In [0]:
 %sh
 rm -r /dbfs/hyperopt_lab
 mkdir /dbfs/hyperopt_lab
 wget -O /dbfs/hyperopt_lab/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv

rm: cannot remove '/dbfs/hyperopt_lab': No such file or directory
--2024-04-04 12:24:25--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9533 (9.3K) [text/plain]
Saving to: ‘/dbfs/hyperopt_lab/penguins.csv’

     0K .........                                             100% 1.81M=0.005s

2024-04-04 12:24:25 (1.81 MB/s) - ‘/dbfs/hyperopt_lab/penguins.csv’ saved [9533/9533]



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
   
data = spark.read.format("csv").option("header", "true").load("/hyperopt_lab/penguins.csv")
data = data.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"),
                          col("CulmenDepth").astype("float"),
                          col("FlipperLength").astype("float"),
                          col("BodyMass").astype("float"),
                          col("Species").astype("int")
                          )
display(data.sample(0.2))
   
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Island,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
Torgersen,41.1,17.6,182.0,3200.0,0
Torgersen,34.6,21.1,198.0,4400.0,0
Torgersen,38.7,19.0,195.0,3450.0,0
Torgersen,34.4,18.4,184.0,3325.0,0
Biscoe,38.2,18.1,185.0,3950.0,0
Biscoe,37.9,18.6,172.0,3150.0,0
Dream,37.0,16.9,185.0,3000.0,0
Dream,41.1,19.0,182.0,3425.0,0
Dream,37.5,18.9,179.0,2975.0,0
Dream,36.0,17.9,190.0,3450.0,0


Training Rows: 238  Testing Rows: 104


In [0]:
from hyperopt import STATUS_OK
import mlflow
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
   
def objective(params):
    # Train a model using the provided hyperparameter value
    catFeature = "Island"
    numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
    catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
    numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
    numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
    featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
    mlAlgo = DecisionTreeClassifier(labelCol="Species",    
                                    featuresCol="Features",
                                    maxDepth=params['MaxDepth'], maxBins=params['MaxBins'])
    pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, mlAlgo])
    model = pipeline.fit(train)
       
    # Evaluate the model to get the target metric
    prediction = model.transform(test)
    eval = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction", metricName="accuracy")
    accuracy = eval.evaluate(prediction)
       
    # Hyperopt tries to minimize the objective function, so you must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}

In [0]:
from hyperopt import fmin, tpe, hp
   
# Define a search space for two hyperparameters (maxDepth and maxBins)
search_space = {
    'MaxDepth': hp.randint('MaxDepth', 10),
    'MaxBins': hp.choice('MaxBins', [10, 20, 30])
}
   
# Specify an algorithm for the hyperparameter optimization process
algo=tpe.suggest
   
# Call the training function iteratively to find the optimal hyperparameter values
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=6)
   
print("Best param values: ", argmin)

  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:52<04:22, 52.58s/trial, best loss: -0.9519230769230769]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 33%|███▎      | 2/6 [01:32<03:00, 45.03s/trial, best loss: -0.9711538461538461]

...

 67%|██████▋   | 4/6 [02:51<01:22, 41.27s/trial, best loss: -0.9711538461538461]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 83%|████████▎ | 5/6 [03:29<00:40, 40.22s/trial, best loss: -0.9711538461538461]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 6/6 [04:07<00:00, 41.30s/trial, best loss: -0.9711538461538461]
Best param values:  {'MaxBins': 2, 'MaxDepth': 2}


In [0]:
from hyperopt import Trials
   
# Create a Trials object to track each run
trial_runs = Trials()
   
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=3,
  trials=trial_runs)
   
print("Best param values: ", argmin)
   
# Get details from each trial run
print ("trials:")
for trial in trial_runs.trials:
    print ("\n", trial)

In [0]:
data.write.saveAsTable("penguins")